In [29]:
import duckdb

con = duckdb.connect()

con.execute("""
CREATE TABLE learner_activity AS
SELECT * FROM read_csv_auto('../data/raw/learner_activity_raw.csv')
""")


In [30]:
con.execute("""
SELECT COUNT(*) AS total_rows 
FROM learner_activity
""").fetchall()


[(900,)]

In [31]:
%pip install duckdb


Note: you may need to restart the kernel to use updated packages.


In [32]:
con.execute("""
SELECT COUNT(*) AS total_rows
FROM learner_activity
""").fetchall()


[(900,)]

In [33]:
con.execute("""
SELECT COUNT(*) AS invalid_video_progress
FROM learner_activity
WHERE video_progress_pct > 100
   OR video_progress_pct < 0
""").fetchall()


[(103,)]

In [34]:
import pandas as pd

df = pd.read_csv('../data/raw/learner_activity_raw.csv')
df.head()


,user_id,course_id,enrollment_date,module_name,video_progress_pct,time_spent_minutes,assessment_attempts,assessment_score,completion_status,feedback_rating,device_type,access_time
0,U0058,DS101,2024-06-09,Python Basics,NaN,0,3.0,NaN,Y,1.0,mobile,10:30 AM
1,U0288,DS101,2024/06/21,Python Basics,NaN,119,NaN,50.0,N,2.0,mobile,22:15
2,U0173,DS101,2024-06-13,Intro to Data,81.0,159,NaN,NaN,No,NaN,Laptop,9 PM
3,U0186,ML301,2024-06-23,ML Intro,NaN,0,1.0,42.0,N,3.0,Desktop,22:15
4,U0190,DA201,22-06-2024,Intro Analytics,NaN,0,1.0,89.0,N,5.0,Laptop,22:15


In [35]:
df.info()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 900 entries, 0 to 899
Data columns (total 12 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   user_id              900 non-null    object 
 1   course_id            900 non-null    object 
 2   enrollment_date      900 non-null    object 
 3   module_name          900 non-null    object 
 4   video_progress_pct   471 non-null    float64
 5   time_spent_minutes   900 non-null    int64  
 6   assessment_attempts  663 non-null    float64
 7   assessment_score     469 non-null    float64
 8   completion_status    900 non-null    object 
 9   feedback_rating      746 non-null    float64
 10  device_type          900 non-null    object 
 11  access_time          900 non-null    object 
dtypes: float64(4), int64(1), object(7)
memory usage: 84.5+ KB


In [36]:
df['enrollment_date'] = pd.to_datetime(
    df['enrollment_date'],
    errors='coerce'
)


In [37]:
df['video_progress_pct'] = pd.to_numeric(
    df['video_progress_pct'],
    errors='coerce'
)

df['video_progress_pct'] = df['video_progress_pct'].clip(0, 100)



In [38]:
df['time_spent_minutes'] = pd.to_numeric(
    df['time_spent_minutes'],
    errors='coerce'
)


In [39]:
df['assessment_attempts'] = pd.to_numeric(
    df['assessment_attempts'],
    errors='coerce'
).fillna(0).astype(int)

df['assessment_score'] = pd.to_numeric(
    df['assessment_score'],
    errors='coerce'
).fillna(0)


In [40]:
df['completion_status'] = (
    df['completion_status']
    .astype(str)
    .str.strip()
    .str.lower()
    .map({
        'y': 'Yes',
        'yes': 'Yes',
        'n': 'No',
        'no': 'No'
    })
)


In [41]:
df['device_type'] = (
    df['device_type']
    .astype(str)
    .str.strip()
    .str.lower()
)


In [42]:
df['feedback_rating'] = pd.to_numeric(
    df['feedback_rating'],
    errors='coerce'
)


In [43]:
df.isna().sum()


user_id                  0
course_id                0
enrollment_date        608
module_name              0
video_progress_pct     429
time_spent_minutes       0
assessment_attempts      0
assessment_score         0
completion_status        0
feedback_rating        154
device_type              0
access_time              0
dtype: int64

In [44]:
import os
os.getcwd()


'c:\\Users\\Arthi\\Learner-Journey-Funnel-Drop-off-Analysis\\notebooks'

In [45]:
import os

BASE_DIR = os.path.abspath(os.path.join(os.getcwd(), ".."))
CLEANED_PATH = os.path.join(
    BASE_DIR, "data", "cleaned", "learner_activity_cleaned.csv"
)

CLEANED_PATH


'c:\\Users\\Arthi\\Learner-Journey-Funnel-Drop-off-Analysis\\data\\cleaned\\learner_activity_cleaned.csv'

In [46]:
df.to_csv(CLEANED_PATH, index=False)


In [47]:
import pandas as pd
import random
from datetime import datetime, timedelta

random.seed(42)

users = [f"U{str(i).zfill(4)}" for i in range(1, 301)]  # 300 users
courses = ["DS101", "DA201", "ML301"]

modules = {
    "DS101": ["Intro to Data", "SQL Basics", "Python Basics"],
    "DA201": ["Intro Analytics", "EDA Concepts", "Dashboards"],
    "ML301": ["ML Intro", "Regression", "Model Evaluation"]
}

rows = []

for _ in range(900):  # 🔥 900 rows
    user = random.choice(users)
    course = random.choice(courses)
    module = random.choice(modules[course])

    enrollment_date = datetime(2024, 6, 1) + timedelta(days=random.randint(0, 30))
    enrollment_date = enrollment_date.strftime(
        random.choice(["%Y-%m-%d", "%d-%m-%Y", "%Y/%m/%d"])
    )

    progress = random.choice([None, random.randint(-10, 120)])
    time_spent = random.choice([0, random.randint(5, 180)])

    attempts = random.choice([None, 1, 2, 3])
    score = random.choice([None, random.randint(30, 95)])

    completion = random.choice(["Y", "Yes", "N", "No"])
    rating = random.choice([None, 1, 2, 3, 4, 5])
    device = random.choice(["Mobile", "mobile", "Desktop", "desktop", "Laptop"])
    access_time = random.choice(["10:30 AM", "22:15", "9 PM", "14:20"])

    rows.append([
        user, course, enrollment_date, module, progress, time_spent,
        attempts, score, completion, rating, device, access_time
    ])

df_raw = pd.DataFrame(rows, columns=[
    "user_id", "course_id", "enrollment_date", "module_name",
    "video_progress_pct", "time_spent_minutes",
    "assessment_attempts", "assessment_score",
    "completion_status", "feedback_rating",
    "device_type", "access_time"
])

df_raw.to_csv('../data/raw/learner_activity_raw.csv', index=False)

df_raw.shape


(900, 12)